# LSTM nyelvmodell

Az inspiráció nem más, mint  [Andrej Karpathy híres írása](https://karpathy.github.io/2015/05/21/rnn-effectiveness/). 


## Adatbeolvasás

In [ ]:
import numpy as np
import tensorflow as tf
import nltk

from numpy.random import seed
seed(1212)

tf.random.set_random_seed(1234)

nltk.download("brown")

from nltk.corpus import brown

# This can be an important parameter, so be aware of it...
max_seq_length = 15
max_num_of_sents = 57200
# max_num_of_sents = 50 # How many sentences should we read from the corpus (max=57200)

def generate_brown_word_to_id_map():
    """Return a dictionary mapping downcased Brown-words to their ids.
    Numbering starts from 1 since we use 0 for masking (!!!).
    """
    words = set()
    for word in brown.words():
        words.add(word.lower())
    return {word: idx + 1 for idx, word in enumerate(sorted(words))}


class BrownReader:
    """A reader class for the Brown corpus.
    """

    def __init__(self):
        self.word_to_id_map = generate_brown_word_to_id_map()
        self.id_to_word_map = {idx: word for word, idx in self.word_to_id_map.items()}

    def n_words(self):
        return len(self.word_to_id_map)

    def sentence_to_ids(self, sentence):
        """Return the word ids of a sentence.
        """
        return [self.word_to_id_map[word.lower()] for word in sentence]
        
    def sentences(self):
        """Generator yielding features from the Brown corpus.
        """
        return (self.sentence_to_ids(sentence) for sentence in brown.sents())

    def sentence_matrixes(self):
        x = np.zeros((max_num_of_sents, max_seq_length-1))
        y = np.zeros((max_num_of_sents, max_seq_length-1))
        sents = self.sentences()
        for idx, sent in enumerate(sents):
            if idx == max_num_of_sents:
                break
            np_array = np.asarray(sent)
            length  = min(max_seq_length, len(np_array))
            x[idx, :length - 1] = np_array[:length - 1]
            y[idx, :length - 1] = np_array[1:length]
        return x, y


## Modell

### Paraméterek

In [ ]:
br = BrownReader()
n_words = br.n_words()

max_input_length = max_seq_length - 1 # since our x/y input does not contain the last/first element of the sentences

In [ ]:
data_x, data_y = br.sentence_matrixes()

In [ ]:
data_y = np.expand_dims(data_y, -1) # It seems that Keras needs this for the "one-cold" and softmax dims to match

# Feladatok

Alább

In [ ]:
# háló paraméterek

lstm_size = ???
embedding_size = ???


### Háló

In [ ]:
# Import:
# Importáld a megfelelő rétegeket
# Gondolj bele, hogy a hálóban az első réteg egy "beágazás" kell majd legyen
# Ne feledd behozni a funkcionális vagy szekvenciális API-nak megfelelő "fő" osztályt
# Adott esetben az optimalizálót
# Nem külömben a "bakcendet", hogy jó gyakorlat szerint reseteld a gráfot
# valmint a ritka, kategorikus keresztentrópiát, mint veszteségfüggvényt

# jó gyakorlat: reseteld a gráfot!

# Model
########
# Építs modellt!
# Bemeneti réteggel kezdd, aminek az inputja egy maximális szöveghosszt jelző vektor. 
# Meg ugye hogy akár vektor, akár nem, a shape az tuple...
# Ezt kövesse egy beágyazás réteg.
# FIGYELEM: 
# 1. a szélessége a szavak száma plusz 1
# 2. mérete paraméterben adott, lásd fent
# 3. bemenet hossza: legnagyobb bemenet mérete -1
# 4. a nulla értékek maszkolandóak benne
# FELADAT: Ezeket indokold meg, miért?

# Következő KÉT rétegben LSTM-ek legyenek.
# Ugye ahhoz, hogy egymásra építhetőek legyenek, nem csak a szekvencia végi predikcióikat kell visszaadják
# Erre van valahol egy csinos paraméter... ;-)

# Végül pedig egy fully connected layerrel és softmaxxal projektáljuk a kimenetet.
# Mennyi a szélessége? (Segítség: ha a fentieket jól megindokoltad, akkor már tudod. ;-)

# Végül példányosítsuk a modellt!

model.summary()


### Hiba, optimalizáció és modelfordítás

In [ ]:
# Loss 

loss = ??? # One-hot enkódolt kimenetünk van. Mit is használunk?

# Optimizer
optimizer = ??? #Ízlés szerint...
 
# Compilation
#############

???

### Tréning

Előállítjuk a tréningadatot:

És trénelünk:

In [ ]:
# Illesszük az adatra a modellt. Használjunk 10% validációt
# - nyelvmodellnél ez nem olyan lényeges
# Használhatjuk a Keras beépített validációs splitjét.
# Adjunk meg reális batch méretet!

## Demó 1: Következő szó

In [ ]:
# Prediction
############

def str_to_input(s):
    """Convert a string to appropriate model input.
    """
    words = [x.lower() for x in s.split()[:max_input_length]]
    ids = [br.word_to_id_map[word] for word in words]
    ids_array = np.asarray(ids)
    length = min(max_input_length, len(ids_array))
    result = np.zeros((1, max_input_length))
    result[0, :length] = ids_array[:length]
    return result, length
    

while True:
    s = input("\nEnter a few starting words of a sentence or <return> to stop: ")
    if s == "":
        break
    else:
        try:
            x, length = str_to_input(s)
            predictions = model.predict(x)
            probs = predictions[0][length - 1]
            most_probable = np.argmax(probs)
            print("Predicted next word:", br.id_to_word_map[most_probable])
        except KeyError:
            print("Unknown words -- please try again!")


## Demó 2: Mondatok hasonlósága

Először egy függvényt definiálunk, amely előállítja a rejtett LSTM állapotokat egy inputból:

In [ ]:
input_layer = model.get_layer("input_1")
lstm_2_layer = model.get_layer("lstm_1")

hidden_states_fun = K.function([input_layer.input],[lstm_2_layer.output])

def get_embedding(x, timestep):
    """Return the hidden state associated with an input at the given timestep.
    """
    hidden_states = hidden_states_fun([x])[0]
    return hidden_states[0, timestep]


In [ ]:
def cos_sim(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

while True:
    s1 = input("\nEnter the first sentence or <return> to quit: ")
    if s1 == "": break
    s2 = input("\nEnter the second sentence: ")
    try:
        x1, l1 = str_to_input(s1)
        x2, l2 = str_to_input(s2)
        e1 = get_embedding(x1, l1-1)
        e2 = get_embedding(x2, l2-1)
        print("The cosine similarity between the two sentences is", cos_sim(e1, e2))
    except KeyError:
        print("Unknown words -- please try again!")

## Demó 3: Mini search engine

A Spotify által kiadott [Annoy](https://github.com/spotify/annoy) library segítségével felhasználjuk a tréningezett modellt arra, hogy a Brown corpus minden mondatához egy LSTM belső reprezentációjából származó vektort rendeljünk, majd szomszédossági kereséssel mini keresőt hozzunk létre.

In [ ]:
def brown_sent_to_input(ids):
  ids_array = np.asarray(ids)
  length = min(max_input_length, len(ids_array))
  result = np.zeros((1, max_input_length))
  result[0, :length] = ids_array[:length]
  return result, length

In [ ]:
sentlist = list(br.sentences())

In [ ]:
!pip install annoy

In [ ]:
INDEX_COVERAGE_PERCENT = 1.0 #How much of the corpus you want ot index? 1.0 means whole, 0.5 means half.
NEAREST_NEIGHBOR_NUM = 5

In [ ]:
from annoy import AnnoyIndex
from tqdm import tqdm

index = AnnoyIndex(512, metric="angular")

for i in tqdm(range(len(int(sentlist*INDEX_COVERAGE_PERCENT)))):
  inputs,length = brown_sent_to_input(sentlist[i])
  vector = get_embedding(inputs, length-1)
  index.add_item(i,vector)

print("Building index...")
index.build(100)
print("Index done, ready to query!")

In [ ]:
def print_brown_index(sentences, indices):
  for i in indices:
    word_ids_list = sentences[i]
    for j in word_ids_list:
      print(br.id_to_word_map[j]+" ", end='')
    print()

    

In [ ]:
while True:
  query = input("\nEnter the query or <return> to quit: ")
  if query == "": break
  try:
    in_ids, length = str_to_input(query)
    in_vector = get_embedding(in_ids, length-1)
    nearest_sentence_indices = index.get_nns_by_vector(in_vector, NEAREST_NEIGHBOR_NUM)
    #print("nearest indices:", nearest_sentence_indices)
    print_brown_index(sentlist, nearest_sentence_indices)

  except KeyError:
    print("Unknown words -- please try again!")